In [1]:
import numpy as np
import os
import pickle
from time import time
from datetime import datetime

In [2]:
import pulse2percept as p2p
import p2pspatial

2018-01-14 23:33:41,034 [pulse2percept] [INFO] Welcome to pulse2percept


In [3]:
subject = '12-005'
amplitude = 2.0
electrodes = ['A01']
random_state = 42
n_folds = 5

In [4]:
t_start = time()
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
filename = 'crossval-swarm_%s_%s.pickle' % (subject, now)
print(filename)

crossval-swarm_12-005_2018-01-14_23-33-41.pickle


In [5]:
rootfolder = os.path.join(os.environ['SECOND_SIGHT_DATA'], 'shape')
X, y = p2pspatial.load_data(rootfolder, subject=subject, electrodes=electrodes,
                            amplitude=amplitude, random_state=random_state,
                            single_stim=True, verbose=True)
print(X.shape, y.shape)
if len(X) == 0:
    raise ValueError('no data found')

Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20091028\1.5xTh 20Hz 0.45ms 250ms
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20091028\5xTh 20Hz 0.45ms 250ms
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100127\1.25xTh 20Hz 0.45ms 250ms set A
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100127\1.5xTh 20Hz 0.45ms 250ms set A
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100127\1.5xTh 20Hz 0.45ms 250ms set B
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100127\2xTh 20Hz 0.45ms 250ms set A
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100127\2xTh 20Hz 0.45ms 250ms set B
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100203\2xTh 20Hz 0.45ms 250ms set A
Found 35 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100428\1.5xTh 20Hz 0.45ms 250ms set A
Found 40 samples in C:\Users\mbeyeler\data\seco

In [6]:
model_params = {'sampling': 200,
                'csmode': 'gaussian',
                'sensitivity_rule': 'decay',
                'thresh': 1.0}
regressor = p2pspatial.SpatialModelRegressor(**model_params)

In [7]:
search_params = {'decay_const': (0.001, 10),
                 'cswidth': (10, 1000),
                 'implant_x': (-1500, 1500),
                 'implant_y': (-1000, 1000),
                 'implant_rot': np.deg2rad((-75, -15))}
pso_options = {'max_iter': 100,
               'min_func': 0.1}
pso = p2pspatial.model_selection.ParticleSwarmOptimizer(
    regressor, search_params, **pso_options
)

In [8]:
fit_params = {'loc_od_x': 15.5,
              'loc_od_y': 1.5,
              'use_ofl': True,
              'use_persp_trafo': False}
y_test, y_pred, best_params = p2pspatial.model_selection.crossval_predict(
    pso, X, y, fit_params=fit_params, n_folds=n_folds)

Fold 1 / 5
No constraints given.
mean loss: 71.0836261038 55.3989191099


KeyboardInterrupt: 

In [ ]:
print("Done in %.3fs" % (time() - t_start))

In [ ]:
specifics = {'subject': subject,
             'amplitude': amplitude,
             'electrodes': electrodes,
             'n_folds': n_folds,
             'regressor': regressor,
             'optimizer': pso,
             'model_params': model_params,
             'search_params': search_params,
             'fit_params': fit_params,
             'now': now,
             'random_state': random_state}
pickle.dump((y_test, y_pred, best_params, specifics), open(filename, 'wb'))
print('Dumped data to %s' % filename)

In [ ]:
def meow(Xy):
    (_, Xrow), (_, yrow) = Xy
    print(Xrow['amp'])
    print(yrow['area'])
    return 0

In [ ]:
np.allclose(img.shape, img.shape)